# Assignment 6
Group D
## Exercise 1: Fuzzy Clustering
### 1. calculate distances from points to clusters:
|       | $p_1$      | $p_2$ | $p_3$      | $p_4$      |
|-------|------------|-------|------------|------------|
| $c_1$ | $\sqrt{2}$ | 1     | 1          | $\sqrt{2}$ |
| $c_2$ | $\sqrt{5}$ | 2     | $\sqrt{2}$ | 1          |

- applying equation $(2)$ for $f_{2,1}$ and $f_{4,1}$ ($f_{2,2} = 1 - f_{2,1}$):
    - $f_{2,1} = 1/(1+1/4) = 4/5$
    - $f_{4,1} = 1/(1+2) = 1/3$
    - thus we get $F = \begin{pmatrix} 5/7 & 2/7 \\ 4/5 & 1/5 \\ 2/3 & 1/3 \\ 1/3 & 2/3 \end{pmatrix}$

### 2. recalculate cluster centers

In [26]:
import numpy as np

def sumPoints(Fj,p,b):
    s = (0,0)
    for mu in range(len(Fj)):
        s += Fj[mu]**b * p[mu]
    return s

F = np.array([[5/7, 2/7],[4/5, 1/5],[2/3, 1/3],[1/3, 2/3]])
b = 2
p = np.array([(0,0),(1,0),(2,1),(2,2)])

c_1 = 1 / sum(F[:,0]**2) * sumPoints(F[:,0],p,b)
c_2 = 1 / sum(F[:,1]**2) * sumPoints(F[:,1],p,b)
print(c_1)
print(c_2)

[1.64149739 0.78166543]
[3.24872756 2.46115725]


![cluster centers after 1 iteration](c(1).JPG)

### 3. result after 100 iterations
The fuzzy result differs just slightly from the expected crisp clustering result of $c_1(100)=\begin{pmatrix} 0.5 \\ 0\end{pmatrix}$ and $c_2(100)=\begin{pmatrix} 2 \\ 1.5 \end{pmatrix}$

### 4. a)
- i. A higher value for $b$ makes the clusters move towards the origin (0,0). Thus they come closer together and the clustering becomes more ambiguous. The algorithm still works, but it is not very useful anymore.
- ii. The overlap increases.

### 4. b)
$ \lim_{b \to 1+} f_{1,1} ^b = \frac{1}{1+ \bigl( \frac{|| P_1 - C_1 ||^2}{|| P_1 - C_2 ||^2} \bigl) ^\frac{1}{b-1}} = \frac{1}{1+\bigl( \frac{\sqrt{2}}{\sqrt{5}} \bigl) ^\frac{1}{b-1}} = 0$

$ \lim_{b \to 1+} f_{1,2} ^b = \frac{1}{1+ \bigl( \frac{|| P_1 - C_2 ||^2}{|| P_1 - C_1 ||^2} \bigl) ^\frac{1}{b-1}} = \frac{1}{1+\bigl( \frac{\sqrt{5}}{\sqrt{2}} \bigl) ^\frac{1}{b-1}} = 1$

### 4. c)
$ \lim_{b \to \infty} \tilde{f_{1,1}} = 1 - \frac{|| P_1 - C_1 ||^2}{|| P_1 - C_1 ||^2 + || P_1 - C_2 ||^2} = 1 - \frac{\sqrt{2}}{\sqrt{2} + \sqrt{5}} = \frac{5-\sqrt{10}}{3} \approx 0,613 $

$ \lim_{b \to \infty} \tilde{f_{1,2}} = 1 - \frac{|| P_1 - C_2 ||^2}{|| P_1 - C_1 ||^2 + || P_1 - C_2 ||^2} = 1 - \frac{\sqrt{5}}{\sqrt{2} + \sqrt{5}} = \frac{-2+\sqrt{10}}{3} \approx 0,387 $

### 4. d)
The result would be the same except the thickness of the lines. They would be thinner.

## Exercise 2: Crisp vs. Fuzzy $k$-means
- see CrispVsFuzzy.ipynb